<a href="https://colab.research.google.com/github/kzumreen/FoodTrendsPrediction/blob/main/Reddit_Access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install praw pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import praw
import time
import pandas as pd

In [3]:
CLIENT_ID = userdata.get("REDDIT_CLIENT_ID")
CLIENT_SECRET = userdata.get("REDDIT_CLIENT_SECRET")
USER_AGENT = userdata.get("REDDIT_USER_AGENT")

In [4]:
try:
    reddit = praw.Reddit(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        user_agent=USER_AGENT
    )
    print("PRAW connection successful. Operating in read-only mode.")
except Exception as e:
    print(f"An error occurred during PRAW initialization: {e}")
    reddit = None

PRAW connection successful. Operating in read-only mode.


In [5]:
TREND_MAP = {
    'feta-pasta': ['feta pasta', 'baked feta', 'tiktok pasta', 'viral pasta',
                   'feta tomato pasta', 'baked feta tomato'],
    'matcha': ['matcha', 'matcha latte', 'matcha powder', 'matcha tea',
               'ceremonial matcha', 'matcha recipe', 'iced matcha'],
    'dubai-chocolate': ['dubai chocolate', 'knafeh chocolate', 'fix chocolate',
                        'pistachio chocolate', 'viral chocolate', 'kadayif chocolate'],
    'cottage-cheese': ['cottage cheese', 'cottage cheese ice cream',
                       'cottage cheese recipe', 'high protein cottage'],
    'air-fryer': ['air fryer', 'air fried', 'airfryer recipe', 'air fryer cooking'],
    'crumbl': ['crumbl cookie', 'crumbl', 'giant cookie', 'crumbl copycat']
}

In [6]:
# BASELINE (non-trending) foods for comparison
BASELINE_MAP = {
    'pizza': ['pizza', 'pepperoni pizza', 'homemade pizza'],
    'chicken': ['chicken breast', 'grilled chicken', 'chicken recipe'],
    'salad': ['salad', 'caesar salad', 'green salad'],
    'pasta-general': ['pasta', 'spaghetti', 'marinara', 'pasta recipe']
}

In [11]:
TARGET_SUBREDDITS = [
    # Original subreddits
    'food', 'cooking', 'recipes', 'baking', 'streetfood',
    'easyrecipes', 'snacksindia', 'askUK', 'candy', 'chocolate',
    'UAE', 'seriouseats', 'RecipeInspiration', 'unpopularopinion',
    'uk_food', 'cookingtonight', 'pasta', 'foodhack', 'nutrition',
    'vegetarianrecipes', 'foodvideos', 'budgetcooking', 'dessertporn',
    'junkfoodfinds', 'popculturechat', 'tea', 'shittyfoodporn',
    'snacks', 'foodlosangeles', 'stupidfood', 'TikTokTrends',
    'TikTokCringe',

    # NEW ADDITIONS - High-value subreddits for trend analysis
    'TikTokFood',
    'GifRecipes',
    'MealPrepSunday',
    '1200isplenty',
    'EatCheapAndHealthy',
    'ketorecipes',
    'veganrecipes',
    'FoodPorn',
    'HealthyFood',
    'fitmeals',
    'Bento',
    'DessertPorn',
    'IceCreamery',
    'Coffee',
    'CasualConversation'
]

print(f"✓ Defined {len(TREND_MAP)} trend categories")
print(f"✓ Defined {len(BASELINE_MAP)} baseline categories")
print(f"✓ Targeting {len(TARGET_SUBREDDITS)} subreddits")

✓ Defined 6 trend categories
✓ Defined 4 baseline categories
✓ Targeting 47 subreddits


In [12]:
def search_reddit_globally(keywords, time_filter='year', limit=500):
    """
    Search across ALL of Reddit for specific keywords
    This is CRITICAL for finding trend-specific posts
    """
    all_reddit = reddit.subreddit('all')
    posts_data = []

    for keyword in keywords:
        print(f"  → Searching all of Reddit for '{keyword}'...")
        try:
            for submission in all_reddit.search(keyword, time_filter=time_filter, limit=limit):
                posts_data.append({
                    'id': submission.id,
                    'subreddit': str(submission.subreddit),
                    'title': submission.title,
                    'score': submission.score,
                    'num_comments': submission.num_comments,
                    'author': str(submission.author),
                    'url': submission.url,
                    'created_utc': submission.created_utc,
                    'search_keyword': keyword
                })
        except Exception as e:
            print(f"    ✗ Error searching for '{keyword}': {e}")
            continue

        time.sleep(2)  # Rate limiting

    return posts_data


In [14]:
def get_subreddit_top_posts(subreddit_name, limit=200):
    """
    Get top posts from a subreddit (multiple time periods)
    """
    posts_data = []

    try:
        subreddit = reddit.subreddit(subreddit_name)

        # Get top posts from different time periods
        for time_filter in ['year', 'month']:
            for submission in subreddit.top(time_filter=time_filter, limit=limit):
                posts_data.append({
                    'id': submission.id,
                    'subreddit': subreddit_name,
                    'title': submission.title,
                    'score': submission.score,
                    'num_comments': submission.num_comments,
                    'author': str(submission.author),
                    'url': submission.url,
                    'created_utc': submission.created_utc,
                    'search_keyword': 'top_posts'
                })
            time.sleep(1)

    except Exception as e:
        print(f"  ✗ Error fetching from r/{subreddit_name}: {e}")

    return posts_data

In [15]:
def get_subreddit_new_posts(subreddit_name, limit=500):
    """
    Get newest posts from a subreddit
    """
    posts_data = []

    try:
        subreddit = reddit.subreddit(subreddit_name)

        for submission in subreddit.new(limit=limit):
            posts_data.append({
                'id': submission.id,
                'subreddit': subreddit_name,
                'title': submission.title,
                'score': submission.score,
                'num_comments': submission.num_comments,
                'author': str(submission.author),
                'url': submission.url,
                'created_utc': submission.created_utc,
                'search_keyword': 'new_posts'
            })

    except Exception as e:
        print(f"  ✗ Error fetching new posts from r/{subreddit_name}: {e}")

    return posts_data

In [16]:
def remove_duplicates(posts_list):
    """
    Remove duplicate posts based on ID
    """
    seen_ids = set()
    unique_posts = []

    for post in posts_list:
        if post['id'] not in seen_ids:
            seen_ids.add(post['id'])
            unique_posts.append(post)

    return unique_posts


In [17]:
print("=" * 70)
print("PHASE 1: GLOBAL TREND SEARCH")
print("=" * 70)

all_trend_posts = []

# Collect data for each TREND
for trend_id, keywords in TREND_MAP.items():
    print(f"\n📊 Collecting data for: {trend_id.upper()}")

    # Global search across all Reddit
    trend_posts = search_reddit_globally(keywords, time_filter='all', limit=500)

    # Add trend identifier
    for post in trend_posts:
        post['trend_category'] = trend_id
        post['is_baseline'] = False

    all_trend_posts.extend(trend_posts)
    print(f"  ✓ Found {len(trend_posts)} posts for {trend_id}")

    time.sleep(3)  # Rate limiting

# Remove duplicates
all_trend_posts = remove_duplicates(all_trend_posts)

print(f"\n{'='*70}")
print(f"✓ PHASE 1 COMPLETE: Collected {len(all_trend_posts)} unique trend posts")
print(f"{'='*70}")


print("\n" + "=" * 70)
print("PHASE 2: BASELINE FOOD SEARCH")
print("=" * 70)

all_baseline_posts = []

# Collect data for each BASELINE food
for baseline_id, keywords in BASELINE_MAP.items():
    print(f"\n📊 Collecting baseline data for: {baseline_id.upper()}")

    # Global search with smaller limit
    baseline_posts = search_reddit_globally(keywords, time_filter='year', limit=200)

    # Add baseline identifier
    for post in baseline_posts:
        post['trend_category'] = baseline_id
        post['is_baseline'] = True

    all_baseline_posts.extend(baseline_posts)
    print(f"  ✓ Found {len(baseline_posts)} posts for {baseline_id}")

    time.sleep(3)

# Remove duplicates
all_baseline_posts = remove_duplicates(all_baseline_posts)

print(f"\n{'='*70}")
print(f"✓ PHASE 2 COMPLETE: Collected {len(all_baseline_posts)} unique baseline posts")
print(f"{'='*70}")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



PHASE 1: GLOBAL TREND SEARCH

📊 Collecting data for: FETA-PASTA
  → Searching all of Reddit for 'feta pasta'...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



  → Searching all of Reddit for 'baked feta'...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

  → Searching all of Reddit for 'marinara'...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



  → Searching all of Reddit for 'pasta recipe'...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



  ✓ Found 800 posts for pasta-general

✓ PHASE 2 COMPLETE: Collected 2448 unique baseline posts


In [18]:
print("\n" + "=" * 70)
print("PHASE 3: SUBREDDIT-SPECIFIC COLLECTION")
print("=" * 70)

all_subreddit_posts = []

for subreddit_name in TARGET_SUBREDDITS:
    print(f"\n📊 Collecting from r/{subreddit_name}")

    # Get top posts
    top_posts = get_subreddit_top_posts(subreddit_name, limit=100)

    # Get new posts
    new_posts = get_subreddit_new_posts(subreddit_name, limit=200)

    # Combine and mark as subreddit collection
    sub_posts = top_posts + new_posts
    for post in sub_posts:
        post['trend_category'] = 'subreddit_general'
        post['is_baseline'] = False

    all_subreddit_posts.extend(sub_posts)
    print(f"  ✓ Collected {len(sub_posts)} posts from r/{subreddit_name}")

    time.sleep(2)  # Rate limiting

# Remove duplicates
all_subreddit_posts = remove_duplicates(all_subreddit_posts)

print(f"\n{'='*70}")
print(f"✓ PHASE 3 COMPLETE: Collected {len(all_subreddit_posts)} unique subreddit posts")
print(f"{'='*70}")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




PHASE 3: SUBREDDIT-SPECIFIC COLLECTION

📊 Collecting from r/food


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



  ✓ Collected 400 posts from r/food


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

  ✓ Collected 375 posts from r/Coffee


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




📊 Collecting from r/CasualConversation


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



  ✓ Collected 400 posts from r/CasualConversation

✓ PHASE 3 COMPLETE: Collected 13910 unique subreddit posts


In [20]:
# Combine all posts
all_posts = all_trend_posts + all_baseline_posts + all_subreddit_posts

# Remove final duplicates
all_posts = remove_duplicates(all_posts)

# Create DataFrame
df = pd.DataFrame(all_posts)

print(f"\n✓ Total unique posts collected: {len(df)}")
print(f"✓ Date range: {len(df)} posts")

# Display breakdown
print(" Data Breakdown:")
print(df['trend_category'].value_counts())



✓ Total unique posts collected: 22369
✓ Date range: 22369 posts
 Data Breakdown:
trend_category
subreddit_general    13723
matcha                1528
dubai-chocolate       1146
feta-pasta             942
crumbl                 899
cottage-cheese         877
air-fryer              840
pasta-general          751
chicken                560
salad                  559
pizza                  544
Name: count, dtype: int64


In [23]:
df['date'] = pd.to_datetime(df['created_utc'], unit='s')

df = df.sort_values(by='date', ascending=False)

START_YEAR = 2015
df = df[df['date'].dt.year >= START_YEAR].copy()

df['Time_Period'] = df['date'].dt.date.astype('datetime64[ns]')

print(f"✓ Data filtered: {len(df)} posts from {START_YEAR} to present")
print(f"✓ Date range: {df['date'].min()} to {df['date'].max()}")

✓ Data filtered: 22358 posts from 2015 to present
✓ Date range: 2015-01-04 13:42:22 to 2025-10-19 23:59:30


In [25]:
df['Trend_ID'] = 'other'

def create_keyword_pattern(keywords):
    """Create a regex pattern from keyword list"""
    return '|'.join([kw.lower() for kw in keywords])

for trend_id, keywords in TREND_MAP.items():
    pattern = create_keyword_pattern(keywords)
    mask = (df['title'].str.contains(pattern, case=False, na=False)) & \
           (df['Trend_ID'] == 'other')
    df.loc[mask, 'Trend_ID'] = trend_id
    print(f"  ✓ Assigned {mask.sum()} posts to {trend_id}")

for baseline_id, keywords in BASELINE_MAP.items():
    pattern = create_keyword_pattern(keywords)
    mask = (df['title'].str.contains(pattern, case=False, na=False)) & \
           (df['Trend_ID'] == 'other') & \
           (df['is_baseline'] == True)
    df.loc[mask, 'Trend_ID'] = baseline_id
    print(f"  ✓ Assigned {mask.sum()} posts to baseline: {baseline_id}")

print(f"Final Trend ID Distribution:")
print(df['Trend_ID'].value_counts())


  ✓ Assigned 290 posts to feta-pasta
  ✓ Assigned 1441 posts to matcha
  ✓ Assigned 448 posts to dubai-chocolate
  ✓ Assigned 521 posts to cottage-cheese
  ✓ Assigned 636 posts to air-fryer
  ✓ Assigned 642 posts to crumbl
  ✓ Assigned 543 posts to baseline: pizza
  ✓ Assigned 264 posts to baseline: chicken
  ✓ Assigned 482 posts to baseline: salad
  ✓ Assigned 615 posts to baseline: pasta-general
Final Trend ID Distribution:
Trend_ID
other              16476
matcha              1441
crumbl               642
air-fryer            636
pasta-general        615
pizza                543
cottage-cheese       521
salad                482
dubai-chocolate      448
feta-pasta           290
chicken              264
Name: count, dtype: int64


In [26]:
print("\n" + "=" * 70)
print("DATA QUALITY CHECK")
print("=" * 70)

print("\n1. Dataset Shape:")
print(f"   Rows: {df.shape[0]}, Columns: {df.shape[1]}")

print("\n2. Date Range:")
print(f"   Earliest: {df['date'].min()}")
print(f"   Latest: {df['date'].max()}")

print("\n3. Missing Values:")
print(df.isnull().sum())

print("\n4. Numerical Summary:")
print(df[['score', 'num_comments']].describe())

print("\n5. Top 5 Subreddits:")
print(df['subreddit'].value_counts().head())

print("\n6. Sample Data:")
print(df[['title', 'subreddit', 'Trend_ID', 'date', 'score']].head(10))

# Check trend coverage
print("\n7. Trend Post Counts:")
for trend in TREND_MAP.keys():
    count = len(df[df['Trend_ID'] == trend])
    print(f"   {trend}: {count} posts")


DATA QUALITY CHECK

1. Dataset Shape:
   Rows: 22358, Columns: 14

2. Date Range:
   Earliest: 2015-01-04 13:42:22
   Latest: 2025-10-19 23:59:30

3. Missing Values:
id                0
subreddit         0
title             0
score             0
num_comments      0
author            0
url               0
created_utc       0
search_keyword    0
trend_category    0
is_baseline       0
date              0
Time_Period       0
Trend_ID          0
dtype: int64

4. Numerical Summary:
               score  num_comments
count   22358.000000  22358.000000
mean     2205.035513    183.333080
std      8100.881356    626.427009
min         0.000000      0.000000
25%         9.000000      5.000000
50%       112.000000     23.500000
75%       841.000000    102.000000
max    207320.000000  22735.000000

5. Top 5 Subreddits:
subreddit
food              634
shittyfoodporn    494
tea               477
1200isplenty      411
FoodPorn          403
Name: count, dtype: int64

6. Sample Data:
                 

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
df.to_csv('/content/drive/MyDrive/reddit_food_trends_complete.csv', index=False)

In [32]:
for trend in TREND_MAP.keys():
    trend_df = df[df['Trend_ID'] == trend]
    if len(trend_df) > 0:
        filename = f'/content/drive/MyDrive/reddit_{trend}_data.csv'
        trend_df.to_csv(filename, index=False)
        print(f"Saved: reddit_{trend}_data.csv ({len(trend_df)} posts)")



Saved: reddit_feta-pasta_data.csv (290 posts)
Saved: reddit_matcha_data.csv (1441 posts)
Saved: reddit_dubai-chocolate_data.csv (448 posts)
Saved: reddit_cottage-cheese_data.csv (521 posts)
Saved: reddit_air-fryer_data.csv (636 posts)
Saved: reddit_crumbl_data.csv (642 posts)
